Set cuda

In [1]:
import torch
import os
import sys
device=0 # No other value works. known bug in Jupyter.

if torch.cuda.is_available():
    torch.cuda.set_device(device)

imports

In [2]:
%env WANDB_NOTEBOOK_NAME=JUPYTER

import wandb
import math

import os
from myhelpers import TrialStatistics
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from tqdm import tqdm
from tqdm.auto import trange
wandb.login()

env: WANDB_NOTEBOOK_NAME=JUPYTER


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find JUPYTER
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)


True

sweep params

In [3]:
# sweep_config = {
#     'method': 'random'
# }

sweep_config = {
    'method': 'bayes',
    'early_terminate': {
       'type': 'hyperband',
       'min_iter': 8   
    }
}
metric = {
    'name': 'test_fine_f1',
    'goal': 'maximize'   
    }
sweep_config['metric'] = metric
COUNT=100

In [4]:
project="HGNN_CIFAR_Adaptivesmoothing_Tripletloss_Hyperp_SGD"
params = {
    "image_path":  {
        'value':"cifar-100-python"#"Curated4/Easy_50"
    },
    "suffix":{
        'value': None
    },
    
    # dataset
    "img_res": {
        'value': 32 #448
    }, 
    "augmented": {
        'value': False
    }, 

    # training
    "batchSize": {
          'value': 128 #[8, 16, 23, 64, 128, 256] # 8, 16, 64, 128
        }, 
#     "learning_rate":{
#         'distribution': 'log_uniform',
#         'min': math.log(0.00005),
#         'max': math.log(0.5),
#       },
    "learning_rate": { 'value': 0.001},
    "numOfTrials":{
        'value': 1
    },
    "fc_layers": {
          'value': 1
        },
#     "fc_layers": {'value': 2},
    "modelType":{
        'value':"BB"
    }, #BB DISCO DSN HGNN HGNN_add HGNN_cat
#     "lambda": {
#         'distribution': 'log_uniform',
#         'min': math.log(0.005),
#         'max': math.log(0.5),
#       },
    "lambda":{
        'value': 0
    },
#     "unsupervisedOnTest": {
#         'value': False
#     },
    "tl_model": {
        'value': 'CIFAR'#'ResNet18'
    },
    "link_layer": {
        'value': 'avgpool'
    },
#     "link_layer":  {
#         'values': ["avgpool","layer3"]
#     }, 
    
#     "adaptive_smoothing": {
#         'values': [True,False]
#     },
    "adaptive_smoothing": {
        'value': True
    },
#     "adaptive_lambda": {
#         'value': 0.015
#     },
    "adaptive_lambda": {
        'distribution': 'log_uniform',
        'min': math.log(0.001),
        'max': math.log(0.5),
      }, 
    "adaptive_alpha": {
        'min': 0.01,
        'max': 0.99,
      }, 
    
    "tripletEnabled": {'value':True},
    "tripletSamples": {'value': 2},
    "tripletSelector": {'values': ["semihard", 'hard', 'random']},
    "tripletMargin": {'values': [0.002, 0.02, 0.2, 2, 20]},
#     "adaptive_alpha": {
#         'value': 0.5
#     },
    
#     "noSpeciesBackprop":{
#         'values': [True,False]
#     },
#     "noSpeciesBackprop": {
#         'value': False
#     },
    "phylogeny_loss":{
        'value': False#[False, "MSE", "KLDiv"] # False, "MSE", "KLDiv"
    },
#     "phylogeny_loss_epsilon": {
#         'distribution': 'log_uniform',
#         'min': math.log(0.001),
#         'max': math.log(1),
#       }, 
    "phylogeny_loss_epsilon": {
        'value': 0.03
    },
}

sweep_config['parameters'] = params

train function

In [5]:

# python3 HGNN/code/HGNN/HGNN/train/train.py --cuda=6 --name="biology_paper_curated4_Medium_backprop_experiment" -- -- --detailed

experimentsPath = "/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = project

def train(config=None):
    run = wandb.init()

#     try:
    experiment_params=wandb.config if config is None else config
    experiment_params = dict(experiment_params)
    config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
    print(experiment_params)
    experimentHash =TrialStatistics.getTrialName(experiment_params)

    # load images 
    experimentPathAndName = os.path.join(experimentsPath, experimentName)
    if experiment_params['image_path'] == 'cifar-100-python':
        datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
    else:
        datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
    datasetManager.updateParams(config_parser.fixPaths(experiment_params))
    train_loader, validation_loader, test_loader = datasetManager.getLoaders()
    architecture = {
        "fine": len(train_loader.dataset.csv_processor.getFineList()),
        "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
    }

    # Loop through n trials
    for i in trange(experiment_params["numOfTrials"], desc="trial"):
        modelName = getModelName(experiment_params, i)
        trialName = os.path.join(experimentPathAndName, modelName)
        trialHash = TrialStatistics.getTrialName(experiment_params, i)

        row_information = {
            'experimentName': experimentName,
            'modelName': modelName,
            'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
            'experimentHash': experimentHash,
            'trialHash': trialHash
        }
        row_information = {**row_information, **experiment_params} 
        print(row_information)


        # Train/Load model
        model = CNN.create_model(architecture, experiment_params, device=device)

        if os.path.exists(CNN.getModelFile(trialName)):
            print("Model {0} found!".format(trialName))
        else:
            CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=True)

    run.finish()
#     except:
#         print('terminated! Error:', sys.exc_info()[0])

Call sweeper!

In [6]:
# You can call with your own configs if you want:
# train(config={...})

In [7]:
# Or call the sweeper and let it do the magic!
sweep_id = wandb.sweep(sweep_config, project=project)

Create sweep with ID: q0gbehkn
Sweep URL: https://wandb.ai/mndhamod/HGNN_CIFAR_Adaptivesmoothing_Tripletloss_Hyperp_SGD/sweeps/q0gbehkn


In [ ]:
wandb.agent(sweep_id, function=train, count=COUNT)
# !CUDA_VISIBLE_DEVICES=1 wandb agent k7cnzayi --count 100 #train.py from program?

wandb: Agent Starting Run: a4qjcggv with config:
wandb: 	adaptive_alpha: 0.575701917398489
wandb: 	adaptive_lambda: 0.055949818864499096
wandb: 	adaptive_smoothing: True
wandb: 	augmented: False
wandb: 	batchSize: 128
wandb: 	fc_layers: 1
wandb: 	image_path: cifar-100-python
wandb: 	img_res: 32
wandb: 	lambda: 0
wandb: 	learning_rate: 0.001
wandb: 	link_layer: avgpool
wandb: 	modelType: BB
wandb: 	numOfTrials: 1
wandb: 	phylogeny_loss: False
wandb: 	phylogeny_loss_epsilon: 0.03
wandb: 	suffix: None
wandb: 	tl_model: CIFAR
wandb: 	tripletEnabled: True
wandb: 	tripletMargin: 2
wandb: 	tripletSamples: 2
wandb: 	tripletSelector: hard
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find JUPYTER
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'adaptive_alpha': 0.575701917398489, 'adaptive_lambda': 0.055949818864499096, 'adaptive_smoothing': True, 'augmented': False, 'batchSize': 128, 'fc_layers': 1, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.001, 'link_layer': 'avgpool', 'modelType': 'BB', 'numOfTrials': 1, 'phylogeny_loss': False, 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'tripletEnabled': True, 'tripletMargin': 2, 'tripletSamples': 2, 'tripletSelector': 'hard'}
Creating dataset...
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Creating dataset... Done.
Loading saved indices...
Creating loaders...
Creating loaders... Done.


{'experimentName': 'HGNN_CIFAR_Adaptivesmoothing_Tripletloss_Hyperp_SGD', 'modelName': 'models/6e519725309dddbc0e19df34a017dd37d1184d0bda82ec82547c6f03', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': '8fd8abde45ef4ad766e42fae75c0b7de7526561a408fdb16451df6e9', 'trialHash': '6e519725309dddbc0e19df34a017dd37d1184d0bda82ec82547c6f03', 'adaptive_alpha': 0.575701917398489, 'adaptive_lambda': 0.055949818864499096, 'adaptive_smoothing': True, 'augmented': False, 'batchSize': 128, 'fc_layers': 1, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.001, 'link_layer': 'avgpool', 'modelType': 'BB', 'numOfTrials': 1, 'phylogeny_loss': False, 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'tripletEnabled': True, 'tripletMargin': 2, 'tripletSamples': 2, 'tripletSelector': 'hard'}


iteration:   0%|          | 0/500 [00:00<?, ?it/s]

layer4 is not in ['conv1', 'bn1', 'relu', 'layer1', 'layer2', 'layer3', 'avgpool']
Training started...


...

{'adaptive_alpha': 0.8792429964187144, 'adaptive_lambda': 0.0020293948242825267, 'adaptive_smoothing': True, 'augmented': False, 'batchSize': 128, 'fc_layers': 1, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.001, 'link_layer': 'avgpool', 'modelType': 'BB', 'numOfTrials': 1, 'phylogeny_loss': False, 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'tripletEnabled': True, 'tripletMargin': 2, 'tripletSamples': 2, 'tripletSelector': 'semihard'}
Creating dataset...
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Creating dataset... Done.
Loading saved indices...
Creating loaders...
Creating loaders... Done.


{'experimentName': 'HGNN_CIFAR_Adaptivesmoothing_Tripletloss_Hyperp_SGD', 'modelName': 'models/a2bb1e314fc7ca6e6c53074b4303455088441103c000371682f5cfc7', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': '53f5650754cd6316e3d0b05ae6f5bca797e2119473de99d1c1fdac76', 'trialHash': 'a2bb1e314fc7ca6e6c53074b4303455088441103c000371682f5cfc7', 'adaptive_alpha': 0.8792429964187144, 'adaptive_lambda': 0.0020293948242825267, 'adaptive_smoothing': True, 'augmented': False, 'batchSize': 128, 'fc_layers': 1, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.001, 'link_layer': 'avgpool', 'modelType': 'BB', 'numOfTrials': 1, 'phylogeny_loss': False, 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'tripletEnabled': True, 'tripletMargin': 2, 'tripletSamples': 2, 'tripletSelector': 'semihard'}


iteration:   0%|          | 0/500 [00:00<?, ?it/s]

layer4 is not in ['conv1', 'bn1', 'relu', 'layer1', 'layer2', 'layer3', 'avgpool']
Training started...


iteration:   0%|          | 2/500 [06:16<18:14:09, 131.83s/it, min_val_loss=1.04, train=0.999, val=0.997, val_loss=3.63]